In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re
import time
import requests


def extraer_sku_y_gramos(url):
    """Usa Selenium para abrir la página y obtener SKU y gramos/ml (Shopify carga con JS)."""
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Esperar a que cargue el bloque principal del producto
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "p.product__sku, p.custom_product__text"))
        )

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # --- SKU ---
        sku = "N/A"
        p_sku = soup.find("p", class_="product__sku")
        if p_sku:
            texto = p_sku.get_text(strip=True)
            match = re.search(r"SKU[:\s]*([A-Za-z0-9\-]+)", texto)
            if match:
                sku = match.group(1)
            else:
                sku = texto

        # --- Gramos / ml ---
        gramos = "N/A"
        p_gramos = soup.find("p", class_="custom_product__text product__text")
        if p_gramos:
            texto_g = p_gramos.get_text(strip=True)
            match_g = re.search(r"(\d+(?:[.,]\d+)?\s*(?:g|ml|gr|GR|ML))", texto_g, re.IGNORECASE)
            if match_g:
                gramos = match_g.group(1).replace(",", ".").lower()

        driver.quit()
        return sku, gramos

    except Exception as e:
        print(f"⚠️ Error en {url}: {e}")
        try:
            driver.quit()
        except:
            pass
        return "N/A", "N/A"


def rostro_ro():
    """Extrae TODOS los productos de la categoría 'Cuidado de la Piel' desde Nala Rumania (https://nala.ro/collections/ten)."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.ro/collections/ten")

    # Esperar carga de productos
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # --- Scroll dinámico ---
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.ro" + enlace.get("href")

        # ====== 🔍 Extracción robusta del nombre ======
        nombre = None
        nombre_tag = item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # --- Precio ---
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio = precio_match.group() + " £"

# --- Ingrediente clave (detectado en rumano y traducido al español) ---
        ingrediente_match = re.search(
        r"(căpșuni|zmeură|struguri|avocado|aloe vera|argan|unt de shea|lavandă|mușețel|castravete|cocos|lămâie|piersică|mango|grapefruit|vanilie|cafea|rodie|mentă|trandafiri|cânepă|pepene verde|portocală|ananas|coacăze|cireșe|gălbenele|jojoba)",
        nombre,
        re.IGNORECASE
        )

        ingrediente = "N/A"
        if ingrediente_match:
            ingrediente_rumano = ingrediente_match.group(1).lower()
            ingrediente = {
                "căpșuni": "Fresa", "zmeură": "Frambuesa", "struguri": "Uvas", "avocado": "Aguacate",
            "aloe vera": "Aloe Vera", "argan": "Argán", "unt de shea": "Karité", "lavandă": "Lavanda",
            "mușețel": "Manzanilla", "castravete": "Pepino", "cocos": "Coco", "lămâie": "Limón",
            "piersică": "Melocotón", "mango": "Mango", "grapefruit": "Pomelo", "vanilie": "Vainilla",
            "cafea": "Café", "rodie": "Granada", "mentă": "Menta", "trandafiri": "Rosas", "cânepă": "Cáñamo",
            "pepene verde": "Sandía", "portocală": "Naranja", "ananas": "Piña", "coacăze": "Grosella negra",
            "cireșe": "Cereza", "gălbenele": "Caléndula", "jojoba": "Jojoba"
            }.get(ingrediente_rumano, "N/A")


        # --- Clasificación por categoría ---
        nombre_lower = nombre.lower()
        if "exfoliant" in nombre_lower and "buze" in nombre_lower:
            categoria = "Exfoliante Labial"
        elif "exfoliant" in nombre_lower:
            categoria = "Exfoliante Facial"
        elif "ochi" in nombre_lower:
            categoria = "Contorno de Ojos"
        elif "crema" in nombre_lower:
            categoria = "Crema Facial"
        elif "serum" in nombre_lower or "sérum" in nombre_lower:
            categoria = "Serum Facial"
        elif "set" in nombre_lower:
            categoria = "Set Facial"
        elif "spf" in nombre_lower or "protecție solară" in nombre_lower:
            categoria = "Protector Solar Facial"
        elif "balsam" in nombre_lower:
            categoria = "Bálsamo"
        elif "spumă" in nombre_lower:
            categoria = "Espuma Limpiadora"
        elif "gel" in nombre_lower:
            categoria = "Gel Limpiador"
        elif "tonic" in nombre_lower:
            categoria = "Tónico Facial"
        elif "mască" in nombre_lower:
            categoria = "Mascarilla Facial"
        elif "micelar" in nombre_lower:
            categoria = "Agua Micelar Limpiador"
        elif "lapte" in nombre_lower:
            categoria = "Leche Limpiador Facial"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",   # se completará después
            "gramos/ml": "N/A",
            "nombre": nombre,
            "categoria_general": "Rostro",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave":ingrediente,
            "país": "Rumania",
            "url": url_producto,
        })

    # === Extraer SKUs y gramos/ml en paralelo ===
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")
    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("£", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
        )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]   # Extrae el número
        .astype(float)                        # Convierte a decimal
        .round()                              # Redondea si hay decimales
        .astype("Int64")                      # Convierte a entero permitiendo NaN
    )

    return df




In [11]:
df_rostro_ro = rostro_ro()
df_rostro_ro

Total productos detectados en la página: 116
Productos encontrados en HTML: 116
Total productos finales: 116


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,70482,11,Unt Buze - Nucă de Cocos & VanilieAdaugă la .,Rostro,Otro,19.9,Coco,Rumania,https://nala.ro/products/balsam-buze-nuca-coco...
1,64836,11,Unt Buze - CireșeAdaugă la .,Rostro,Otro,19.9,Cereza,Rumania,https://nala.ro/products/balsam-buze-cirese
2,30788,120,Loțiune după bărbierit - Ulei Avocado & Ienupă...,Rostro,Otro,34.9,Aguacate,Rumania,https://nala.ro/products/lotiune-dupa-barbieri...
3,70472,50,Exfoliant Facial - Exfoliere Profundă - Cafea ...,Rostro,Exfoliante Facial,29.9,Café,Rumania,https://nala.ro/products/exfoliant-facial-exfo...
4,64827,15,Cremă Ochi Iluminatoare - Cearcăne & Pungi - C...,Rostro,Contorno de Ojos,39.9,N/A,Rumania,https://nala.ro/products/crema-ochi-iluminatoa...
...,...,...,...,...,...,...,...,...,...
111,3152,<NA>,Rutina TOTAL LIFTAdaugă la .,Rostro,Otro,239.9,N/A,Rumania,https://nala.ro/products/rutina-total-lift
112,3153,<NA>,Rutina HYDRA LIFT PROAdaugă la .,Rostro,Otro,229.9,N/A,Rumania,https://nala.ro/products/rutina-hydra-lift-pro
113,3154,<NA>,Rutina HYDRA INTENSE PROAdaugă la .,Rostro,Otro,209.9,N/A,Rumania,https://nala.ro/products/rutina-hydra-intense-pro
114,3155,<NA>,Rutina LIFT OILAdaugă la .,Rostro,Otro,139.9,N/A,Rumania,https://nala.ro/products/rutina-lift-oil


In [12]:
def cabello_ro():
    """Extrae TODOS los productos de la categoría 'Cabello' desde Nala Rumania (https://nala.ro/collections/par)."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.ro/collections/par")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # Scroll dinámico
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.ro" + enlace.get("href")

        # ====== 🔍 Extracción robusta del nombre ======
        nombre = None
        nombre_tag = item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)})
        if nombre_tag and nombre_tag.get_text(strip=True):
            nombre = nombre_tag.get_text(strip=True)

        if not nombre:
            hidden_name = item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE))
            if hidden_name:
                nombre = hidden_name.get_text(strip=True)

        if not nombre:
            nombre = enlace.get_text(strip=True)

        if not nombre:
            nombre = "N/A"
        # =====================================================

        # --- Precio ---
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        precio = "N/A"
        if precio_tag:
            precio_texto = precio_tag.get_text(strip=True)
            precio_match = re.search(r"\d+,\d{2}", precio_texto)
            if precio_match:
                precio = precio_match.group() + " £"

        # --- Ingrediente clave (detectado en rumano y traducido al español) ---
        ingrediente_match = re.search(
            r"(granada|lemongrass|miel|cafeina|camomila|violeta absolut|vanilie|aloe|argan|karité|jojoba|calendula|mușețel|avocado|cânepă|trandafiri|lavandă|mentă|cocos|lămâie|piersică|ananas|mango|castravete|grapefruit|struguri|portocală|spirulină|salvie|migdale|rozmarin|lime|algă|flori de portocal|violeta|lămâiță|ceai verde|ghimbir|vitamina e|keratină|proteine vegetale|ulei de măsline|ceapă|borojó)",
            nombre,
            re.IGNORECASE
        )


        ingrediente = "N/A"
        if ingrediente_match:
            ingrediente_rumano = ingrediente_match.group(1).lower()
            ingrediente = {
                "granada": "Granada", "lemongrass": "Lemongrass", "miel": "Miel", "cafeina": "Cafeína",
                "camomila": "Camomila", "violeta absolut": "Violeta Absoluto", "vanilie": "Vainilla",
                "aloe": "Aloe", "argan": "Argán", "karité": "Karité", "jojoba": "Jojoba",
                "calendula": "Caléndula", "mușețel": "Manzanilla", "avocado": "Aguacate", "cânepă": "Cáñamo",
                "trandafiri": "Rosas", "lavandă": "Lavanda", "mentă": "Menta", "cocos": "Coco",
                "lămâie": "Limón", "piersică": "Melocotón", "ananas": "Piña", "mango": "Mango",
                "castravete": "Pepino", "grapefruit": "Pomelo", "struguri": "Uvas", "portocală": "Naranja",
                "spirulină": "Espirulina", "salvie": "Salvia", "migdale": "Almendra", "rozmarin": "Romero",
                "lime": "Lima", "algă": "Algas", "flori de portocal": "Flor de Naranja", "violeta": "Violeta",
                "lămâiță": "Limoncillo", "ceai verde": "Té Verde", "ghimbir": "Jengibre", "vitamina e": "Vitamina E",
                "keratină": "Queratina", "proteine vegetale": "Proteínas Vegetales", "ulei de măsline": "Aceite de Oliva",
                "ceapă": "Cebolla", "borojó": "Borojó"
            }.get(ingrediente_rumano, "N/A")

        # --- Clasificación por categoría ---
        nombre_lower = nombre.lower()
        if "șampon solid" in nombre_lower:
            categoria = "Champú Sólido"
        elif "șampon" in nombre_lower:
            categoria = "Champú"
        elif "tratament" in nombre_lower:
            categoria = "Tratamiento Capilar"
        elif "mască" in nombre_lower:
            categoria = "Mascarilla Capilar"
        elif "balsam" in nombre_lower:
            categoria = "Acondicionador"
        elif "tonic" in nombre_lower:
            categoria = "Tónico Capilar"
        elif "ulei" in nombre_lower:
            categoria = "Aceite Capilar"
        elif "set" in nombre_lower:
            categoria = "Set Capilar"
        elif "spray" in nombre_lower:
            categoria = "Spray Capilar"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",
            "gramos/ml": "N/A",
            "nombre": nombre,
            "categoria_general": "Cabello",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente,
            "país": "Rumania",
            "url": url_producto,
        })

    # === Extraer SKUs y gramos/ml en paralelo ===
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except Exception:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")

    # === Convertir precio y gramos/ml a columnas numéricas ===
    df["precio"] = (
        df["precio"]
        .str.replace("£", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]
        .astype(float)
        .round()
        .astype("Int64")
    )

    return df


In [13]:
df_cabello_ro = cabello_ro()
df_cabello_ro 

Total productos detectados en la página: 57
Productos encontrados en HTML: 57
Total productos finales: 45


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,30692,200,Șampon Păr Gras - MușețelAdaugă la .,Cabello,Champú,39.9,Manzanilla,Rumania,https://nala.ro/products/sampon-par-gras-musetel
1,64422,200,"Șampon protector Păr Vopsit - Extract Orez, An...",Cabello,Champú,29.9,N/A,Rumania,https://nala.ro/products/sampon-protector-par-...
2,70455,58,Șampon solid Păr Deteriorat - GardenieAdaugă la .,Cabello,Champú Sólido,24.9,N/A,Rumania,https://nala.ro/products/sampon-solid-par-dete...
3,70454,58,Șampon solid Păr Gras - LămâițăAdaugă la .,Cabello,Champú Sólido,24.9,Limoncillo,Rumania,https://nala.ro/products/sampon-solid-par-gras...
4,70456,58,Șampon solid Scalp Sensibil - CăpșuniAdaugă la .,Cabello,Champú Sólido,24.9,N/A,Rumania,https://nala.ro/products/sampon-solid-scalp-se...
5,64240,200,"Șampon Stimulare și Fortifiere - Cafeină, Pipe...",Cabello,Champú,39.9,N/A,Rumania,https://nala.ro/products/sampon-stimulare-cres...
6,64307,200,Șampon Violet Păr Blond - Extract Absolut Viol...,Cabello,Champú,49.9,Limón,Rumania,https://nala.ro/products/sampon-violet-par-blo...
7,70436,200,Șampon Volum și Strălucire - Sare de la Marea ...,Cabello,Champú,39.9,Espirulina,Rumania,https://nala.ro/products/sampon-volum-si-stral...
8,64287,200,"Balsam Păr Blond - Nucă de Cocos, Rozmarin, Lă...",Cabello,Acondicionador,39.9,Coco,Rumania,https://nala.ro/products/balsam-par-blond-ulei...
9,30700,200,Balsam Păr Gras - MușețelAdaugă la .,Cabello,Acondicionador,24.9,Manzanilla,Rumania,https://nala.ro/products/balsam-par-gras-musetel


In [14]:
def corporal_ro():
    """Extrae TODOS los productos de la categoría 'Corporal' en Nala Rumania."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.ro/collections/ingrijire-corp")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # Scroll dinámico
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.ro" + enlace.get("href")

        # Nombre
        nombre = (
            item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)}) or
            item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE)) or
            enlace
        )
        nombre = nombre.get_text(strip=True) if nombre else "N/A"
        nombre_lower = nombre.lower()

        # Precio
        precio = "N/A"
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        if precio_tag:
            match = re.search(r"\d+,\d{2}", precio_tag.get_text(strip=True))
            if match:
                precio = f"{match.group()} £"

        # Ingrediente clave (rumano → español)
        ingrediente_match = re.search(
            r"(căpșuni|zmeură|struguri|avocado|aloe vera|argan|unt de shea|lavandă|mușețel|castravete|cocos|lămâie|piersică|mango|grapefruit|vanilie|cafea|rodie|mentă|trandafiri|cânepă|pepene verde|portocală|ananas|coacăze|cireșe|gălbenele|violeta|migdale|scorțișoară|miere|ciocolată|caramel|liliac|flori de portocal|ceai verde|ghimbir|lemongrass|rozmarin|ulei de măsline)",
            nombre,
            re.IGNORECASE
        )

        ingrediente = "N/A"
        if ingrediente_match:
            ingrediente_rumano = ingrediente_match.group(1).lower()
            ingrediente = {
                "căpșuni": "Fresa", "zmeură": "Frambuesa", "struguri": "Uvas", "avocado": "Aguacate",
                "aloe vera": "Aloe Vera", "argan": "Argán", "unt de shea": "Karité", "lavandă": "Lavanda",
                "mușețel": "Manzanilla", "castravete": "Pepino", "cocos": "Coco", "lămâie": "Limón",
                "piersică": "Melocotón", "mango": "Mango", "grapefruit": "Pomelo", "vanilie": "Vainilla",
                "cafea": "Café", "rodie": "Granada", "mentă": "Menta", "trandafiri": "Rosas", "cânepă": "Cáñamo",
                "pepene verde": "Sandía", "portocală": "Naranja", "ananas": "Piña", "coacăze": "Grosella negra",
                "cireșe": "Cereza", "gălbenele": "Caléndula", "violeta": "Violeta", "migdale": "Almendra",
                "scorțișoară": "Canela", "miere": "Miel", "ciocolată": "Chocolate", "caramel": "Caramelo",
                "liliac": "Lila", "flori de portocal": "Flor de Naranjo", "ceai verde": "Té Verde",
                "ghimbir": "Jengibre", "lemongrass": "Lemongrass", "rozmarin": "Romero", "ulei de măsline": "Aceite de Oliva"
            }.get(ingrediente_rumano, "N/A")

        # Clasificación corporal
        if "mâini" in nombre_lower:
            categoria = "Crema de Manos"
        elif "spray" in nombre_lower:
            categoria = "Spray Corporal"
        elif "gel" in nombre_lower:
            categoria = "Gel Corporal"
        elif "bomba" in nombre_lower:
            categoria = "Bomba de Baño"
        elif "crema" in nombre_lower:
            categoria = "Crema Corporal"
        elif "ulei" in nombre_lower:
            categoria = "Aceite Corporal"
        elif "exfoliant" in nombre_lower:
            categoria = "Exfoliante Corporal"
        elif "apă" in nombre_lower and "parfumată" in nombre_lower:
            categoria = "Bruma"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",
            "gramos/ml": "N/A",
            "nombre": nombre,
            "categoria_general": "Corporal",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente,
            "país": "Rumania",
            "url": url_producto,
        })

    # Extraer SKU y gramos/ml
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")

    # Convertir precio y gramos/ml a columnas numéricas
    df["precio"] = (
        df["precio"]
        .str.replace("£", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]
        .astype(float)
        .round()
        .astype("Int64")
    )

    return df



In [15]:
df_corporal_ro = corporal_ro()
df_corporal_ro

Total productos detectados en la página: 154
Productos encontrados en HTML: 154
⚠️ Error en https://nala.ro/products/unt-corp-piper-rosu-ambra: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff633eee8e5+80021]
	GetHandleVerifier [0x0x7ff633eee940+80112]
	(No symbol) [0x0x7ff633c7060f]
	(No symbol) [0x0x7ff633cc8854]
	(No symbol) [0x0x7ff633cc8b1c]
	(No symbol) [0x0x7ff633d1c927]
	(No symbol) [0x0x7ff633cf126f]
	(No symbol) [0x0x7ff633d1968a]
	(No symbol) [0x0x7ff633cf1003]
	(No symbol) [0x0x7ff633cb95d1]
	(No symbol) [0x0x7ff633cba3f3]
	GetHandleVerifier [0x0x7ff6341adc7d+2960429]
	GetHandleVerifier [0x0x7ff6341a7f3a+2936554]
	GetHandleVerifier [0x0x7ff6341c8977+3070247]
	GetHandleVerifier [0x0x7ff633f083ce+185214]
	GetHandleVerifier [0x0x7ff633f0fe1f+216527]
	GetHandleVerifier [0x0x7ff633ef7b24+117460]
	GetHandleVerifier [0x0x7ff633ef7cdf+117903]
	GetHandleVerifier [0x0x7ff633eddbb8+11112]
	BaseThreadInitThunk [0x0x7ffc14ebe8d7+23]
	RtlUserThreadStart [0x0x7ffc1662c53c+44]

Total prod

,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,35247,150,Apă Parfumată Corp & Păr - Ananas & Lemn Santa...,Corporal,Bruma,29.9,Piña,Rumania,https://nala.ro/products/apa-parfumata-corp-pa...
1,64276,150,Apă Parfumată Corp & Păr - CăpșuniAdaugă la .,Corporal,Bruma,29.9,Fresa,Rumania,https://nala.ro/products/apa-parfumata-corp-ca...
2,35216,150,Apă Parfumată Corp & Păr - Granita de LămâieAd...,Corporal,Bruma,24.9,Limón,Rumania,https://nala.ro/products/apa-parfumata-corp-pa...
3,64277,150,Apă Parfumată Corp & Păr - PapayaAdaugă la .,Corporal,Bruma,29.9,N/A,Rumania,https://nala.ro/products/apa-parfumata-corp-pa...
4,64278,150,Apă parfumată Corp & Păr - PomeloAdaugă la .,Corporal,Bruma,29.9,N/A,Rumania,https://nala.ro/products/apa-parfumata-corp-pa...
...,...,...,...,...,...,...,...,...,...
149,70479,150,Ulei Uscat Corp - Lemn de SantalAdaugă la .,Corporal,Aceite Corporal,39.9,N/A,Rumania,https://nala.ro/products/ulei-uscat-corp-lemn-...
150,74264,150,Ulei Uscat Corp - Mușcată & Portocală AmarăAda...,Corporal,Aceite Corporal,39.9,Naranja,Rumania,https://nala.ro/products/ulei-uscat-corp-musca...
151,70481,150,Ulei Uscat Corp - PapayaAdaugă la .,Corporal,Aceite Corporal,39.9,N/A,Rumania,https://nala.ro/products/ulei-uscat-corp-papaya
152,64894,150,Șerbet Corp - GrapefruitAdaugă la .,Corporal,Otro,39.9,Pomelo,Rumania,https://nala.ro/products/serbet-corp-grapefruit-1


In [16]:
def ducha_ro():
    """Extrae TODOS los productos de la categoría 'Ducha y Baño' en Nala Rumania."""

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.get("https://nala.ro/collections/dus-si-baie")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.grid__item"))
    )

    # Scroll dinámico
    last_count = 0
    stable_rounds = 0
    while stable_rounds < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "li.grid__item")
        if len(products) == last_count:
            stable_rounds += 1
        else:
            stable_rounds = 0
        last_count = len(products)

    print(f"Total productos detectados en la página: {last_count}")

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    items = soup.select("li.grid__item")
    print(f"Productos encontrados en HTML: {len(items)}")

    productos = []
    categorias = defaultdict(int)

    for item in items:
        enlace = item.find("a", href=re.compile("/products/"))
        if not enlace:
            continue
        url_producto = "https://nala.ro" + enlace.get("href")

        # Nombre
        nombre = (
            item.find(attrs={"class": re.compile("(title|heading|card__title)", re.IGNORECASE)}) or
            item.find("span", class_=re.compile("visually-hidden", re.IGNORECASE)) or
            enlace
        )
        nombre = nombre.get_text(strip=True) if nombre else "N/A"
        nombre_lower = nombre.lower()

        # Precio
        precio = "N/A"
        precio_tag = item.find("div", class_=re.compile("price", re.IGNORECASE))
        if precio_tag:
            match = re.search(r"\d+,\d{2}", precio_tag.get_text(strip=True))
            if match:
                precio = f"{match.group()} £"

        # Ingrediente clave (rumano → español)
        ingrediente_match = re.search(
            r"(căpșuni|zmeură|struguri|avocado|aloe vera|argan|unt de shea|lavandă|mușețel|castravete|cocos|lămâie|piersică|mango|grapefruit|vanilie|cafea|rodie|mentă|trandafiri|cânepă|pepene verde|portocală|ananas|coacăze|cireșe|gălbenele|violeta|migdale|scorțișoară|miere|ciocolată|caramel|liliac|flori de portocal|ceai verde|ghimbir|lemongrass|rozmarin|ulei de măsline|sare de mare|lapte|măr|scorțișoară și măr)",
            nombre,
            re.IGNORECASE
        )


        ingrediente = "N/A"
        if ingrediente_match:
            ingrediente_rumano = ingrediente_match.group(1).lower()
            ingrediente = {
                "căpșuni": "Fresa", "zmeură": "Frambuesa", "struguri": "Uvas", "avocado": "Aguacate",
                "aloe vera": "Aloe Vera", "argan": "Argán", "unt de shea": "Karité", "lavandă": "Lavanda",
                "mușețel": "Manzanilla", "castravete": "Pepino", "cocos": "Coco", "lămâie": "Limón",
                "piersică": "Melocotón", "mango": "Mango", "grapefruit": "Pomelo", "vanilie": "Vainilla",
                "cafea": "Café", "rodie": "Granada", "mentă": "Menta", "trandafiri": "Rosas", "cânepă": "Cáñamo",
                "pepene verde": "Sandía", "portocală": "Naranja", "ananas": "Piña", "coacăze": "Grosella negra",
                "cireșe": "Cereza", "gălbenele": "Caléndula", "violeta": "Violeta", "migdale": "Almendra",
                "scorțișoară": "Canela", "miere": "Miel", "ciocolată": "Chocolate", "caramel": "Caramelo",
                "liliac": "Lila", "flori de portocal": "Flor de Naranjo", "ceai verde": "Té Verde",
                "ghimbir": "Jengibre", "lemongrass": "Lemongrass", "rozmarin": "Romero", "ulei de măsline": "Aceite de Oliva",
                "sare de mare": "Sal marina", "lapte": "Leche", "măr": "Manzana", "scorțișoară și măr": "Canela y Manzana"
            }.get(ingrediente_rumano, "N/A")


        # Clasificación por categoría
        if "ulei de duș" in nombre_lower:
            categoria = "Aceite de Ducha"
        elif "mâini" in nombre_lower:
            categoria = "Jabón de Manos"
        elif "duș" in nombre_lower or "gel" in nombre_lower:
            categoria = "Gel de Ducha"
        elif "bomba" in nombre_lower:
            categoria = "Bomba de Baño"
        elif "cremă" in nombre_lower:
            categoria = "Crema de Baño"
        elif "spumă" in nombre_lower:
            categoria = "Espuma de Baño"
        elif "sare" in nombre_lower:
            categoria = "Sal de Baño"
        elif "lapte" in nombre_lower:
            categoria = "Leche de Baño"
        elif "natural" in nombre_lower:
            categoria = "Jabón Natural"
        else:
            categoria = "Otro"

        categorias[categoria] += 1

        productos.append({
            "SKU": "N/A",
            "gramos/ml": "N/A",
            "nombre": nombre,
            "categoria_general": "Ducha y Baño",
            "categoria": categoria,
            "precio": precio,
            "ingrediente_clave": ingrediente,
            "país": "Rumania",
            "url": url_producto,
        })

    # Extraer SKU y gramos/ml
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extraer_sku_y_gramos, p["url"]): i for i, p in enumerate(productos)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                sku, gramos = future.result()
                productos[idx]["SKU"] = sku
                productos[idx]["gramos/ml"] = gramos
            except:
                productos[idx]["SKU"] = "N/A"
                productos[idx]["gramos/ml"] = "N/A"

    df = pd.DataFrame(productos)
    print(f"Total productos finales: {len(df)}")

    # Convertir precio y gramos/ml a columnas numéricas
    df["precio"] = (
        df["precio"]
        .str.replace("£", "", regex=False)
        .str.replace(",", ".", regex=False)
        .str.strip()
        .astype(float)
    )

    df["gramos/ml"] = (
        df["gramos/ml"]
        .str.extract(r"(\d+(?:\.\d+)?)")[0]
        .astype(float)
        .round()
        .astype("Int64")
    )

    return df

In [17]:
df_ducha_ro = ducha_ro()
df_ducha_ro


Total productos detectados en la página: 108
Productos encontrados en HTML: 108
Total productos finales: 108


,SKU,gramos/ml,nombre,categoria_general,categoria,precio,ingrediente_clave,país,url
0,35711,100,Săpun Natural - MerișoareAdaugă la .,Ducha y Baño,Jabón Natural,12.90,N/A,Rumania,https://nala.ro/products/sapun-natural-merisoare
1,35704,100,Săpun Natural - Boabe picante de TonkaAdaugă la .,Ducha y Baño,Jabón Natural,12.90,N/A,Rumania,https://nala.ro/products/sapun-natural-boabe-p...
2,35483,125,Bombă Baie Efervescentă - Afine & StruguriAdau...,Ducha y Baño,Otro,12.90,Uvas,Rumania,https://nala.ro/products/bomba-baie-efervescen...
3,35605,200,Gel de duș - Pară & Pepene RoșuAdaugă la .,Ducha y Baño,Gel de Ducha,19.90,N/A,Rumania,https://nala.ro/products/gel-de-dus-para-pepen...
4,35599,200,Gel de duș - Afine & StruguriAdaugă la .,Ducha y Baño,Gel de Ducha,19.90,Uvas,Rumania,https://nala.ro/products/gel-de-dus-afine-stru...
...,...,...,...,...,...,...,...,...,...
103,65035,200,Gel de duș - Flori de PortocalAdaugă la .,Ducha y Baño,Gel de Ducha,9.90,Flor de Naranjo,Rumania,https://nala.ro/products/gel-de-dus-flori-de-p...
104,64381,200,Gel de duș Exfoliant Hidratant - GrapefruitAda...,Ducha y Baño,Gel de Ducha,29.90,Pomelo,Rumania,https://nala.ro/products/gel-dus-exfoliant-hid...
105,64282,150,Ulei de duș - GrapefruitAdaugă la .,Ducha y Baño,Aceite de Ducha,29.90,Pomelo,Rumania,https://nala.ro/products/ulei-dus-grapefruit-1
106,30759,100,Săpun Natural - Struguri RoșiiAdaugă la .,Ducha y Baño,Jabón Natural,6.45,Uvas,Rumania,https://nala.ro/products/sapun-natural-strugur...


In [18]:
df_combinado = pd.concat([df_cabello_ro, df_corporal_ro , df_ducha_ro, df_rostro_ro], ignore_index=True)

df_combinado.to_csv("nala_ro.csv", index=False)